# HDBSCAN Class

We are going to build a class for doing HDBSCAN. We want the class to relatively closely mirror sklearn's interface for clustering classes. We will also need to build up a set of useful background tools.

## HDBSCAN

We can break the HDBSCAN algorithm down into a series of steps. 

### Mutual reachability graph

The first step is the creation of the 'mutual reachability graph' -- this modifies the distance matrix to include a notion of density based on a parameter `min_points`, such that points have a minimal distance to other points that is at least the distance required to have `min_points` other points within that minimal distance:

Build the mutual reachability graph from a distance matrix. We are operating with the following definitions per 'Campello, Moulavi, Sander':

**Core Distance**: The core distance of an object $x_p \in X$ with respect to $m_{\textrm{pts}}$ is the distance from $x_p$ to its $m_{\textrm{pts}}$-nearest neighbour.

**Mutual Reachability Distance**: The mutual reachability distance between two objects $x_p$ and $x_q$ in $X$ with respect to $m_{\textrm{pts}}$ is defined as $d_{\textrm{mreach}} = \max\{d_{\textrm{core}}(x_p), d_{\textrm{core}}(x_q), d(x_p, x_q)\}$.

### Single linkage clustering

The second step is to apply single linkage clustering to the mutual reachability graph. We can either implement out own single linkage clustering (so as to support nonsymmetric dissimilarity matrices) or use `fastcluster`. For now the implementation uses `fastcluster` -- the only catch is that we need to convert the mutuual reachability matrix into the single array of upper triangular entries that `scipy.spatial.distance.pdist` outputs.

### Relabelling the tree

The third step is to relabel and condense the cluster tree based on a parameter `min_cluster_size`. Any branch that contains less than `min_cluster_size` points is deemed to be points "falling out of an existing cluster" rather than the splitting of a cluster in two. We therefore label the larger child cluster of the splitting with the parent's id and esnure we record at what $\lambda = \frac{1}{\textrm{distance}}$ values the other points "fell out" of the cluster. As we do this throughout the whole tree the number of clusters reduces dramatically. From this relabelled tree we can then generate a table

| Parent id | Child id |  Lambda | Child size |
|----------:|---------:|--------:|-----------:|
|          0|         1|   0.1432|          50|
|        ...|       ...|      ...|         ...|

by flattening the resulting tree out to be in the same format provided by `scipy.cluster.hierarchy` and `fastcluster`. This will then be useful in the next stage when we compute the "stability" of clusters.

### Cluster stability computation

We assign "stability" values to each of the clusters in the relabelled/condensed tree (note that this is significantly fewer then the original single linkage tree. This stability value will then be used to determine which clusters will be selected -- essentially finding a (potentially varying) cut through the cluster hierarchy. We define the stability of the $i^{\textrm{th}}$ cluster $S(C_i)$ as follows:

$S(C_i) = \Sigma_{x_j \in C_i}{(\lambda_{max}(x_j, C_i) - \lambda_{min}(C_i)) }$ <BR>
$= \Sigma_{x_j \in C_i}{(1/{\epsilon_{min}(x_j,C_i)} - 1/{\epsilon_{max}(C_i)})}$

where

* $C_i$ is the $ith$ cluster
* $x_j \in C_i$ represents the point contained in cluster $C_i$
* $\lambda = 1/\epsilon$ is the density threshold.  Increase density threshold decreases cluster sizes.
* $\lambda_{max}(x_j, C_i)$ is the density level beyond which x_j no longer belongs to cluster $C_i$
* $\lambda_{min}(C_i)$ is the density level at which cluster $C_i$ first appears

### Selecting stable clusters


We start by importing a slew of useful libraries which we will be making use of.

In [1]:
import pandas as pd
import numpy as np
import fastcluster as fc
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets as data

%matplotlib inline
sns.set_context("poster")

For condensing tree it is useful to have a tree node class that has the relevant properties:
* An id (not unique per node since we're doing relabelling)
* The distance at which the child nodes split off from their parent.
* A list of children -- we don't require a binary tree as we may have many individual leaves splitting off at the same distance value.
* A boolean denoting whether the node is a leaf node or not.

In [ ]:
class CondensedTreeNode:    
    def __init__(self, id, dist, children, size, is_leaf):
        self.id = id
        self.dist = dist
        self.children = children
        self.child_size = size
        self.is_leaf = is_leaf
    
    def add_child(self, child):
        self.children.append(child)

    def __repr__(self):
        return '<Node object at %s>' % (
            hex(id(self))
            )
        
    def __str__(self):
        return "ID: %d, Lambda %d, Number of Children %d, " \
               "Number of children %d, Leaf node: %s" % (self.id, self.dist, len(self.children), 
                                                         self.child_size, self.is_leaf)

For condensing the tree we need to recurse through the tree. At each stage we check if either the right or left branch of the original tree is smaller than `min_cluster_size`. If it is smaller then we need to add all the leaves as children that split off at the given distance. Furthermore if either the right or the left is smaller than `min_cluster_size` then we want to retain the id of the node and pass it on to the larger of the two children (since we have "points falling out of the cluster" rather than a split forming new clusters).

In [19]:
def get_leaves(tree):
    """Consume a tree object and return a list of leaf nodes"""
    return tree.pre_order(lambda x: x)

def condense_tree(tree, min_cluster_size=10, next_id=0):
        
    #Verbose assert
    if tree.count == 0:
        print("Invalid input: Null tree")
        result = Node(-1, -1, [], -1, False)
        return result
    elif tree.count == 1:
        #Passed in a single node. only that node
        result = Node(-1, tree.dist, [], 1, True)
        return result
        
    result = CondensedTreeNode(next_id, tree.dist, [], tree.left.count + tree.right.count, 0)
        
    #If the left node is too small, add a leaf
    if tree.left.count <= min_cluster_size:
        leaves = get_leaves(tree.left)
        for leaf in leaves:
            result.add_child(CondensedTreeNode(-(leaf.id + 1), tree.left.dist, [], 1, True))
    elif tree.right.count <= min_cluster_size:
        child, next_id = condense_tree(tree.left, min_cluster_size, next_id)
        result.add_child(child)
    else:
        child, next_id = condense_tree(tree.left, min_cluster_size, next_id + 1)
        result.add_child(child)
            
    #If the right node is too small, add a leaf
    if tree.right.count <= min_cluster_size:
        leaves = get_leaves(tree.right)
        for leaf in leaves:
            result.add_child(CondensedTreeNode(-(leaf.id + 1), tree.right.dist, [], 1, True))
    elif tree.left.count <= min_cluster_size:
        child, next_id = condense_tree(tree.right, min_cluster_size, next_id)
        result.add_child(child)        
    else:
        child, next_id = condense_tree(tree.right, min_cluster_size, next_id + 1)
        result.add_child(child)
        
    return result, next_id
    

We can flatten the tree with another recursive function. Since we need to produce a dataframe at the end we will need an outer function and an inner recursive function. A short function generating all the child information for each node can then be called within the recursive function, and we extend the flattened result with the new child information, and pass the result back to the caller (which extends with this information and so on).

In [3]:
def flatten_node(tree_node):
    return [(tree_node.id, x.id, 1.0/tree_node.dist, x.child_size) for x in tree_node.children
             if tree_node.id != x.id]

def flatten_tree_recursion(tree):
    if tree.is_leaf:
        return []
    result = flatten_node(tree)
    for subtree in tree.children:
        result.extend(flatten_tree_recursion(subtree))
    return result

def flatten_tree(tree):
    result = flatten_tree_recursion(tree)
    return pd.DataFrame(result, columns=("parent","child","lambda","child_size"))

The stability of a cluster is $\sum (\lambda_{\textrm{death}} - \lambda_{\textrm{birth}})$. We can compute this for each row of the dataframe output by flatten tree if we have $\lambda_{\textrm{birth}}$ values for each row. To generate that we need to find the minimum $\lambda$ value seen (so we group by child id and take the min of the `lambda` column per group). We can then join that to the main dataframe, and compute a stability value for each row. The total stability for each cluster is then the group by parent id sum of stability values. We can then normalise that by the size of the cluster (so as not to bias large clusters).

In [ ]:
def stability(row):
    return (row["lambda_death"] - row["lambda_birth"]) * row["child_size"]

def compute_stability(cluster_tree):
    births = cluster_tree.groupby("child").min()[["lambda"]]
    births_and_deaths = cluster_tree.join(births, on="parent", lsuffix="_death", rsuffix="_birth")
    births_and_deaths["stability"] = births_and_deaths.apply(stability, axis=1)
    return births_and_deaths.groupby("parent")[["stability"]].sum() / \
            pd.DataFrame(births_and_deaths.parent.value_counts(), columns=["stability"])
    

In [4]:
def stability_score(tree_node, stability_table):
    node_stability = stability_table.loc[tree_node.id][0]
    child_stability = sum(max(stability_score(x, stability_table)) for x in tree_node.children if not x.is_leaf)
    return (node_stability, child_stability)

def get_clusters(tree_node, stability_table, results={}):
    tree_node.score = stability_score(tree_node, stability_table)
    if tree_node.score[0] > tree_node.score[1]:
        tree_node.is_cluster = True
        cluster_id = max(results.keys()) + 1 if results.keys() else 0 
        results[cluster_id] = tree_node.points
    else:
        tree_node.is_cluster = False
    if not tree_node.is_cluster:
        for node in tree_node.children:
            if not node.is_leaf:
                get_clusters(node, stability_table, results)
    return results

def get_leaf_point_ids(tree):
    results = []
    for node in tree.children:
        if node.is_leaf:
            results.append(-(node.id - 1))
        else:
            results.extend(get_leaf_point_ids(node))
    return results

def reduce_tree(tree):
    result = CondensedTreeNode(tree.id, 0, [], 0, 0)
    result.points = get_leaf_point_ids(tree)
    
    children_to_process = tree.children[:]
    
    for child in children_to_process:
        if child.is_leaf:
            continue
        if child.id == tree.id:
            children_to_process.extend(child.children)
        else:
            result.children.append(reduce_tree(child))
            
    return result

Since `fastcluster` assumes that any multidimensional array passed to it is an $N\times p$ feature space and proceeds to pass it to a distance calculator we need to reduce our mutual-reachability matrix to something `fastcluster` will recognise as a distance matrix. In this case that is the raw output of `scipy.spatial.distance.pdist` which is a single-dimensional array that contains the upper triangular part of a distance matrix (that is therefore assumed to be symmetric). This is simple enough to generate from a full distance matrix, so we'll provide a utility function to get that done.

In [5]:
def distance_matrix_to_pdist(matrix):
    result = []
    for index, row in enumerate(distance_matrix):
        result.append(row[index:])
    return np.hstack(result)

Now we come to the main class. The is straightforward: we set `min_cluster_size` and `min_points` (and setting `min_points` to `min_cluster_size` if it was not provided) and the fit method simply takes a distance matrix and performs the various steps described above:

1. Compute the mutual reachability graph
2. Perform single linkage on the resulting mutual reachability graph
3. Condense and relabel the resulting tree according to `min_cluster_size`
4. Compute the cluster stability and extract the most stable clustering that covers the points best.

In [6]:
class HDBSCAN (object):
    
    def __init__(self, min_cluster_size=5, min_points=None):
        self.min_cluster_size = min_cluster_size
        if min_points is None:
            self.min_points = min_cluster_size
        else:
            self.min_points = min_points
        self._mutual_reachability_graph = None
        self._raw_tree = None
        self._tree_frame = None
        
    def _mutual_reachability(self, distance_matrix):
        dim = distance_matrix.shape[0]
        core_distances = np.partition(distance_matrix, self.min_points, axis=0)[self.min_points]
        stage1 = np.where(core_distances > distance_matrix, core_distances, distance_matrix)
        self._mutual_reachability_graph = np.where(core_distances > stage1.T, core_distances.T, stage1.T).T
        return
    
    def _single_linkage(self):
        assert(self._mutual_reachability_graph is not None)
        #self._raw_tree = single_linkage(self._mutual_reachability_graph)
        pdist_array = distance_matrix_to_pdist(self._mutual_reachability_graph)
        self._raw_tree = fc.single(pdist_array)
        return
    
    def _condense_tree(self):
        assert(self._raw_tree is not None)
        base_tree = sch.to_tree(self._raw_tree)
        self._condensed_tree, final_id = condense_tree(base_tree, self.min_cluster_size)
        self._tree_frame = flatten_tree(self._condensed_tree)
        
    def _compute_stable_clusters(self):
        assert(self._tree_frame is not None)
        self._stability_table = compute_stability(self._tree_frame)
        self._reduced_tree = reduce_tree(self._condensed_tree)
        self._cluster_dict = get_clusters(self._reduced_tree, self._stability_table)
    
    def fit(self, distance_matrix):
        self._mutual_reachability(distance_matrix)
        self._single_linkage()
        self._condense_tree()
        self._compute_stable_clusters()
        return self._cluster_dict
        

Now we can do some quick benchmarking to get an idea of performance.

In [7]:
import scipy.spatial.distance as dist

iris = pd.read_csv("iris.csv")
distance_matrix = dist.squareform(dist.pdist(iris.ix[:,:4].as_matrix()))

In [8]:
%%timeit
clusterer = HDBSCAN(10)
clusterer.fit(distance_matrix)

10 loops, best of 3: 76.8 ms per loop


In [9]:
digits = data.load_digits()
digits_distance_matrix = dist.squareform(dist.pdist(digits.data))

In [10]:
%%timeit
clusterer = HDBSCAN(10)
clusterer.fit(digits_distance_matrix)

1 loops, best of 3: 777 ms per loop


In [20]:
%%prun
clusterer = HDBSCAN(10)
clusterer.fit(digits_distance_matrix)

So most of our time is spent in the mutual reachability computation, and within that more than half our time is spent in the (likely very well optimized) partition function used to find the distance of the `min_points`th nearest neighbour distance. It is unlikely we are going to speed that operation up much without some hand-coded C to try and do the job efficiently. 